In [14]:
%reload_ext autoreload
import pandas as pd
from bs4 import BeautifulSoup
import requests
import json
from pipetools import pipe, utils, where
from fastcore.basics import typed
import requests
from fastcore.foundation import L
import pandas as pd
from typing import List
from pathlib import Path
import re
from datetime import datetime, date
from loguru import logger
import os
from p_tqdm import p_map
from tqdm import tqdm_notebook

### Merge fbref and fpl data

In [31]:
fpl_player_stats_df = pd.read_csv("../../data/fpl-data/20210125_GW19_player_stats.csv")

In [32]:
fbref_all = pd.read_csv("../../data/fbref-data/20210125_GW20_fbref.csv")

In [33]:
def get_id(val):
    return val.split("/")[5]

In [34]:
fbref_all["fbref_id"] = fbref_all.profile_url.apply(lambda val: val.split("/")[5])

In [35]:
fbref_all.tail()

,name,profile_url,nationality,position,age,minutes_90s_x,goals,shots_total,shots_on_target,shots_on_target_pct,...,interceptions_y,tackles_won_y,pens_won,pens_conceded,own_goals,ball_recoveries,aerials_won,aerials_lost,aerials_won_pct,fbref_id
468,Martinelli,https://fbref.com/en/players/48a5a5d6/Martinelli,"['br', 'BRA']",FW,19 years and 221 days,1.8,0,7,2,28.6,...,4,1,0,0,0,10,1,2,33.3,48a5a5d6
469,Cédric Soares,https://fbref.com/en/players/839c14e1/Cedric-S...,"['pt', 'POR']",DF,29 years and 147 days,1.1,0,0,0,0.0,...,0,1,0,0,0,9,3,3,50.0,839c14e1
470,Sead Kolašinac,https://fbref.com/en/players/3935e52e/Sead-Kol...,"['ba', 'BIH']",DF,27 years and 219 days,1.0,0,0,0,0.0,...,0,0,0,0,0,12,1,1,50.0,3935e52e
471,Reiss Nelson,https://fbref.com/en/players/c5bdb6e3/Reiss-Ne...,"['eng', 'ENG']",FW,21 years and 046 days,0.8,0,1,0,0.0,...,1,0,0,0,0,4,0,0,0.0,c5bdb6e3
472,Shkodran Mustafi,https://fbref.com/en/players/3f2d59fe/Shkodran...,"['de', 'GER']",DF,28 years and 283 days,0.5,0,0,0,0.0,...,0,1,0,0,0,5,2,0,100.0,3f2d59fe


In [36]:
fpl_player_stats_df.head()

,player_id,name,display_slug,team_id,team,team_slug,position,points_per_game,form,news,...,penalties_missed,saves,bonus,bps,corners_and_indirect_freekicks_order,corners_and_indirect_freekicks_text,direct_freekicks_order,direct_freekicks_text,penalties_order,penalties_text
0,1,Mesut Özil,Özil,1,arsenal,ARS,MID,0.0,0.0,Not included in Arsenal's 25-man Premier Leagu...,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
1,2,Sokratis Papastathopoulos,Sokratis,1,arsenal,ARS,DEF,0.0,0.0,Left the club by mutual consent on 20/1,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
2,3,David Luiz Moreira Marinho,David Luiz,1,arsenal,ARS,DEF,2.0,2.2,NaN,...,0,0,0,108,NaN,NaN,4.0,NaN,NaN,NaN
3,4,Pierre-Emerick Aubameyang,Aubameyang,1,arsenal,ARS,MID,4.2,4.8,Personal Reasons - 75% chance of playing,...,0,0,6,202,NaN,NaN,NaN,NaN,1.0,NaN
4,5,Cédric Soares,Cédric,1,arsenal,ARS,DEF,6.5,2.4,NaN,...,0,0,3,42,NaN,NaN,NaN,NaN,NaN,NaN


In [37]:
fpl_player_stats_df[['player_id', 'name']]

,player_id,name
0,1,Mesut Özil
1,2,Sokratis Papastathopoulos
2,3,David Luiz Moreira Marinho
3,4,Pierre-Emerick Aubameyang
4,5,Cédric Soares
...,...,...
636,615,Lewis Richards
637,616,Christian Marques
638,617,Andreas Söndergaard
639,626,Patrick Cutrone


In [38]:
fbref_names = fbref_all.name.tolist()

In [39]:
fbref_ids = [url.split("/")[5] for url in fbref_all.profile_url.tolist()]

In [40]:
fpl_ids = fpl_player_stats_df.player_id.tolist()

In [41]:
fpl_names = fpl_player_stats_df.name.tolist()

In [42]:
fbref = dict(zip(fbref_ids, fbref_names))

In [43]:
fpl = dict(zip(fpl_ids, fpl_names))

In [44]:
from fuzzywuzzy import fuzz

In [45]:
fuzz.token_sort_ratio("David Luiz Moreira Marinho", "David Luiz")

56

In [49]:
idx = 0
combined_mapper = []
for id_, name in tqdm_notebook(fbref.items()):
    for fid_, fname in fpl.items():
        if fuzz.ratio(name, fname) > 85:
            idx += 1
            combined_mapper.append({"player_id": fid_, "fbref_id": id_})
print(f"Total players processed: {idx}")

<ipython-input-49-c8aafd2afcec>:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for id_, name in tqdm_notebook(fbref.items()):


  0%|          | 0/469 [00:00<?, ?it/s]

Total players processed: 405


In [50]:
mapped_df = pd.DataFrame(combined_mapper)

In [54]:
mapped_df.shape

(405, 2)

In [51]:
mapped_df.columns

Index(['player_id', 'fbref_id'], dtype='object')

In [55]:
fbref_all.shape

(473, 163)

In [57]:
fbref_all[fbref_all.duplicated('fbref_id')]

,name,profile_url,nationality,position,age,minutes_90s_x,goals,shots_total,shots_on_target,shots_on_target_pct,...,interceptions_y,tackles_won_y,pens_won,pens_conceded,own_goals,ball_recoveries,aerials_won,aerials_lost,aerials_won_pct,fbref_id
113,Ross Barkley,https://fbref.com/en/players/3a24769f/Ross-Bar...,"['eng', 'ENG']",MF,27 years and 051 days,0.5,0,1,1,100.0,...,0,0,0,0,0,8,3,1,75.0,3a24769f
264,Theo Walcott,https://fbref.com/en/players/1ecb65be/Theo-Wal...,"['eng', 'ENG']","MF,FW",31 years and 315 days,12.2,2,22,8,36.4,...,6,15,0,0,0,87,4,23,14.8,1ecb65be
374,Robert Snodgrass,https://fbref.com/en/players/9f4647e9/Robert-S...,"['sco', 'SCO']",FW,33 years and 140 days,1.0,0,1,0,0.0,...,1,3,0,0,0,8,1,3,25.0,9f4647e9
434,Ruben Loftus-Cheek,https://fbref.com/en/players/e97fd090/Ruben-Lo...,"['eng', 'ENG']","FW,MF",25 years and 002 days,8.7,1,14,5,35.7,...,7,11,1,0,0,51,17,35,32.7,e97fd090


In [60]:
fbref_all.query('fbref_id == "3a24769f"')

,name,profile_url,nationality,position,age,minutes_90s_x,goals,shots_total,shots_on_target,shots_on_target_pct,...,interceptions_y,tackles_won_y,pens_won,pens_conceded,own_goals,ball_recoveries,aerials_won,aerials_lost,aerials_won_pct,fbref_id
37,Ross Barkley,https://fbref.com/en/players/3a24769f/Ross-Bar...,"['eng', 'ENG']",MF,27 years and 051 days,6.7,2,22,6,27.3,...,2,0,0,0,0,61,10,4,71.4,3a24769f
113,Ross Barkley,https://fbref.com/en/players/3a24769f/Ross-Bar...,"['eng', 'ENG']",MF,27 years and 051 days,0.5,0,1,1,100.0,...,0,0,0,0,0,8,3,1,75.0,3a24769f


In [61]:
combined_one_df = pd.merge(fbref_all, mapped_df, on='fbref_id')

In [63]:
combined_one_df.shape

(409, 164)

In [64]:
pre_final_df = pd.merge(combined_one_df, fpl_player_stats_df, on='player_id')

In [81]:
pre_final_df[['name_x', 'name_y']]

,name_x,name_y
0,Andrew Robertson,Andrew Robertson
1,Roberto Firmino,Roberto Firmino
2,Georginio Wijnaldum,Georginio Wijnaldum
3,Sadio Mané,Sadio Mané
4,Mohamed Salah,Mohamed Salah
...,...,...
404,Joe Willock,Joseph Willock
405,Cédric Soares,Cédric Soares
406,Sead Kolašinac,Sead Kolasinac
407,Reiss Nelson,Reiss Nelson


In [69]:
len(set(fpl_player_stats_df.player_id.tolist()).difference(set(pre_final_df.player_id.tolist())))

236

In [70]:
len(fpl_player_stats_df.player_id.tolist())

641

In [71]:
len(pre_final_df.player_id.tolist())

409

In [73]:
unmapped_fpl_ids = set(fpl_player_stats_df.player_id.tolist()).difference(set(pre_final_df.player_id.tolist()))

In [93]:
unique_team = L(fpl_player_stats_df.team.tolist()).unique()

In [96]:
unique_team.items

['arsenal',
 'aston_villa',
 'brighton',
 'burnley',
 'chelsea',
 'crystal_palace',
 'everton',
 'fulham',
 'leicester',
 'leeds',
 'liverpool',
 'man_city',
 'man_utd',
 'newcastle',
 'sheffield_utd',
 'southampton',
 'spurs',
 'west_brom',
 'west_ham',
 'wolves']

In [120]:
def find_fuzzy_matches(val, fname):
    if fuzz.ratio(val, fname) > 50:
        print(val)
        return val

In [180]:
[{"player_id": pl, "fbref_id": ""} for pl in fpl_player_stats_df.query(f"player_id == {list(unmapped_fpl_ids)}")[['player_id', 'name', 'team']]\
.query("team == 'man_utd'").player_id.tolist()]

[{'player_id': 289, 'fbref_id': ''},
 {'player_id': 292, 'fbref_id': ''},
 {'player_id': 294, 'fbref_id': ''},
 {'player_id': 297, 'fbref_id': ''},
 {'player_id': 299, 'fbref_id': ''},
 {'player_id': 301, 'fbref_id': ''},
 {'player_id': 302, 'fbref_id': ''},
 {'player_id': 304, 'fbref_id': ''},
 {'player_id': 314, 'fbref_id': ''},
 {'player_id': 316, 'fbref_id': ''},
 {'player_id': 317, 'fbref_id': ''},
 {'player_id': 319, 'fbref_id': ''},
 {'player_id': 568, 'fbref_id': ''},
 {'player_id': 575, 'fbref_id': ''},
 {'player_id': 604, 'fbref_id': ''},
 {'player_id': 640, 'fbref_id': ''}]

In [121]:
type(fbref_all.name)

pandas.core.series.Series

In [182]:
unmapped_players = [
    {"player_id": 478, "fbref_id": "8b9ebd03"},
    {"player_id": 3, "fbref_id": "c0e27d1a"},
    {"player_id": 1, "fbref_id": "16380240"},
    {"player_id": 2, "fbref_id": "b682dcf6"},
    {"player_id": 13, "fbref_id": "dc6f5bdd"},
    {"player_id": 19, "fbref_id": "52611641"},
    {"player_id": 22, "fbref_id": "a53649b7"},
    {"player_id": 25, "fbref_id": "5ea8ef20"},
    {"player_id": 26, "fbref_id": "48a5a5d6"},
    {"player_id": 27, "fbref_id": "972aeb2a"},
    {"player_id": 494, "fbref_id": "67ac5bb8"},
    {"player_id": 501, "fbref_id": "c0617e2b"},
    {"player_id": 526, "fbref_id": "47a34c4f"},
    {"player_id": 527, "fbref_id": "136e7649"},
    {"player_id": 547, "fbref_id": "8fa8f8d2"},
    
    {'player_id': 28, 'fbref_id': 'a6de6361'},
     {'player_id': 30, 'fbref_id': 'fe8e6fa1'},
     {'player_id': 32, 'fbref_id': 'a8a3d139'},
     {'player_id': 34, 'fbref_id': ''},
     {'player_id': 35, 'fbref_id': 'bdedffac'},
     {'player_id': 36, 'fbref_id': '4fdf1fd9'},
     {'player_id': 40, 'fbref_id': ''},
     {'player_id': 43, 'fbref_id': 'c07b7c5c'},
     {'player_id': 46, 'fbref_id': '0313a347'},
     {'player_id': 47, 'fbref_id': '5d8d7454'},
     {'player_id': 48, 'fbref_id': 'bfae110f'},
     {'player_id': 49, 'fbref_id': 'dae8623e'},
     {'player_id': 51, 'fbref_id': '04e195ee'},
     {'player_id': 52, 'fbref_id': '6f7d826d'},
     {'player_id': 496, 'fbref_id': '2389cdc2'},
    
    {'player_id': 53, 'fbref_id': 'f2f29dda'},
     {'player_id': 58, 'fbref_id': '7314bba4'},
     {'player_id': 60, 'fbref_id': '47b7e3af'},
     {'player_id': 63, 'fbref_id': '5cec2487'},
     {'player_id': 64, 'fbref_id': 'dc008610'},
     {'player_id': 65, 'fbref_id': 'bb5fbd2b'},
     {'player_id': 72, 'fbref_id': 'ca2bc3df'},
     {'player_id': 73, 'fbref_id': '8ea2227a'},
     {'player_id': 75, 'fbref_id': '17370b95'},
     {'player_id': 503, 'fbref_id': 'c666574f'},
     {'player_id': 528, 'fbref_id': '28300a16'},
     {'player_id': 628, 'fbref_id': 'f230bc30'},
     {'player_id': 631, 'fbref_id': 'a96ba40b'},
     {'player_id': 632, 'fbref_id': '4871d0e8'},
     {'player_id': 633, 'fbref_id': '2ae8e967'},
     
    {'player_id': 84, 'fbref_id': '74e12f1e'},
     {'player_id': 94, 'fbref_id': 'bcfb0142'},
     {'player_id': 520, 'fbref_id': '6a471ee8'},
     {'player_id': 536, 'fbref_id': '6a39bd42'},
     {'player_id': 551, 'fbref_id': '6abefdff'},
     {'player_id': 583, 'fbref_id': 'c73ada83'},
     {'player_id': 591, 'fbref_id': 'fa38722f'},
     {'player_id': 592, 'fbref_id': '4fa62ae4'},
     {'player_id': 612, 'fbref_id': 'bda82fc2'},
     {'player_id': 613, 'fbref_id': 'f8e588df'},
    
    {'player_id': 105, 'fbref_id': '45db685d'},
     {'player_id': 107, 'fbref_id': '79c0821a'},
     {'player_id': 111, 'fbref_id': 'e0bc6fdc'},
     {'player_id': 232, 'fbref_id': 'd2424d1b'},
     {'player_id': 490, 'fbref_id': '86e7deaf'},
     {'player_id': 593, 'fbref_id': '53b781b9'},
     {'player_id': 605, 'fbref_id': 'b105919e'},
    
    {'player_id': 126, 'fbref_id': '76eca7e2'},
     {'player_id': 135, 'fbref_id': 'd7492450'},
     {'player_id': 136, 'fbref_id': '41cc19c2'},
     {'player_id': 143, 'fbref_id': 'eba42d2b'},
     {'player_id': 145, 'fbref_id': '2f8d27bc'},
     {'player_id': 147, 'fbref_id': '9a526299'},
     {'player_id': 522, 'fbref_id': 'a62da625'},
     {'player_id': 534, 'fbref_id': '600ae8cb'},
     {'player_id': 578, 'fbref_id': 'f234c188'},
     {'player_id': 586, 'fbref_id': 'cc70a1d5'},
     {'player_id': 641, 'fbref_id': '50e6dc35'},
    
    {'player_id': 151, 'fbref_id': '939bc634'},
     {'player_id': 154, 'fbref_id': 'ab4b3e35'},
     {'player_id': 158, 'fbref_id': '1961b2aa'},
     {'player_id': 161, 'fbref_id': '6bd603db'},
     {'player_id': 166, 'fbref_id': 'fa031b34'},
     {'player_id': 167, 'fbref_id': '728e9b2e'},
     {'player_id': 170, 'fbref_id': 'c1949191'},
     {'player_id': 502, 'fbref_id': 'cc700722'},
     {'player_id': 596, 'fbref_id': '26d3b5be'},
    
    {'player_id': 171, 'fbref_id': 'af245161'},
     {'player_id': 172, 'fbref_id': '71a08f97'},
     {'player_id': 174, 'fbref_id': 'a48da055'},
     {'player_id': 177, 'fbref_id': '6c679c6d'},
     {'player_id': 179, 'fbref_id': '7eb67dad'},
     {'player_id': 181, 'fbref_id': '0f7533cd'},
     {'player_id': 183, 'fbref_id': '0d85a4e8'},
     {'player_id': 185, 'fbref_id': 'd368feb8'},
     {'player_id': 187, 'fbref_id': '979b811d'},
     {'player_id': 189, 'fbref_id': '10efd0e1'},
     {'player_id': 192, 'fbref_id': '4e229bf9'},
     {'player_id': 585, 'fbref_id': '1f9a3a04'},
    
     {'player_id': 220, 'fbref_id': 'd6f88a1d'},
     {'player_id': 222, 'fbref_id': 'ebd0a048'},
     {'player_id': 223, 'fbref_id': 'd3ce0e89'},
     {'player_id': 226, 'fbref_id': '75a72a99'},
     {'player_id': 237, 'fbref_id': '305a3820'},
    
     {'player_id': 193, 'fbref_id': 'c0398aca'},
     {'player_id': 194, 'fbref_id': '58bc7d18'},
     {'player_id': 196, 'fbref_id': 'c859705e'},
     {'player_id': 199, 'fbref_id': '9f1938ab'},
     {'player_id': 205, 'fbref_id': 'f13d02a6'},
     {'player_id': 207, 'fbref_id': '9a8241af'},
     {'player_id': 208, 'fbref_id': ''},
     {'player_id': 209, 'fbref_id': '55b4f1d6'},
     {'player_id': 212, 'fbref_id': '7d3dc53e'},
     {'player_id': 215, 'fbref_id': '2d82641c'},
     {'player_id': 492, 'fbref_id': '1fb1c435'},
     {'player_id': 532, 'fbref_id': '97d7afbd'},
     {'player_id': 533, 'fbref_id': 'a2c1d8c2'},
     {'player_id': 599, 'fbref_id': '3d46127c'},
     {'player_id': 619, 'fbref_id': '6d646ff7'},
     {'player_id': 620, 'fbref_id': '4654e5f5'},
    
     {'player_id': 242, 'fbref_id': 'f315ca93'},
     {'player_id': 245, 'fbref_id': 'f76e6b4e'},
     {'player_id': 252, 'fbref_id': '7a2e46a8'},
     {'player_id': 253, 'fbref_id': '7f3b388c'},
     {'player_id': 257, 'fbref_id': 'c6dc9ecd'},
     {'player_id': 265, 'fbref_id': ''},
     {'player_id': 531, 'fbref_id': '77e84962'},
     {'player_id': 601, 'fbref_id': 'd5dafedb'},
     {'player_id': 607, 'fbref_id': 'a33ce8e2'},
    
     {'player_id': 266, 'fbref_id': '101da2b5'},
     {'player_id': 267, 'fbref_id': '10072610'},
     {'player_id': 269, 'fbref_id': '0d267745'},
     {'player_id': 277, 'fbref_id': 'bd6351cd'},
     {'player_id': 278, 'fbref_id': '3bb7b8b4'},
     {'player_id': 281, 'fbref_id': '3eb22ec9'},
     {'player_id': 282, 'fbref_id': 'b66315ae'},
     {'player_id': 285, 'fbref_id': ''},
     {'player_id': 286, 'fbref_id': '6434f10d'},
     {'player_id': 537, 'fbref_id': '548672b4'},
     {'player_id': 552, 'fbref_id': 'dc7f8a28'},
     {'player_id': 556, 'fbref_id': '31c69ef1'},
     {'player_id': 595, 'fbref_id': '259fea27'},
     {'player_id': 608, 'fbref_id': '3e1550ee'},
     {'player_id': 609, 'fbref_id': '091c86e2'},
     {'player_id': 622, 'fbref_id': '387e1d35'},
     {'player_id': 623, 'fbref_id': '47327321'},
    
     {'player_id': 326, 'fbref_id': '3a949a25'},
     {'player_id': 327, 'fbref_id': '6eaed4eb'},
     {'player_id': 330, 'fbref_id': '8816329f'},
     {'player_id': 337, 'fbref_id': 'c5942695'},
     {'player_id': 340, 'fbref_id': 'c17bfb65'},
     {'player_id': 341, 'fbref_id': '5343eb4e'},
     {'player_id': 342, 'fbref_id': '603e4f0b'},
     {'player_id': 488, 'fbref_id': 'e32958b7'},
     {'player_id': 606, 'fbref_id': '096d1e60'},
    
     {'player_id': 348, 'fbref_id': 'e7434f04'},
     {'player_id': 350, 'fbref_id': '96cf7b61'},
     {'player_id': 477, 'fbref_id': '636b64cd'},
     {'player_id': 580, 'fbref_id': '424595d5'},
    
    {'player_id': 364, 'fbref_id': '4b511457'},
     {'player_id': 371, 'fbref_id': 'e082af5b'},
     {'player_id': 372, 'fbref_id': 'b0c71810'},
     {'player_id': 374, 'fbref_id': '6d6f09dc'},
     {'player_id': 380, 'fbref_id': 'd484c261'},
     {'player_id': 479, 'fbref_id': '0b33f6ad'},
     {'player_id': 590, 'fbref_id': ''},
     {'player_id': 598, 'fbref_id': ''},
     {'player_id': 624, 'fbref_id': ''},
     {'player_id': 625, 'fbref_id': 'c35e99d8'},
     {'player_id': 634, 'fbref_id': 'f4765c97'},
     {'player_id': 635, 'fbref_id': 'ab7d7882'},
     {'player_id': 636, 'fbref_id': 'c7170b46'},
    
     {'player_id': 384, 'fbref_id': '89d10e53'},
     {'player_id': 390, 'fbref_id': '92e7e919'},
     {'player_id': 392, 'fbref_id': '2b622f01'},
     {'player_id': 393, 'fbref_id': '63d17038'},
     {'player_id': 394, 'fbref_id': 'cea4ee8f'},
     {'player_id': 396, 'fbref_id': '8b04d6c1'},
     {'player_id': 399, 'fbref_id': '6aa3e78b'},
     {'player_id': 401, 'fbref_id': ''},
     {'player_id': 402, 'fbref_id': 'e9971f2d'},
     {'player_id': 404, 'fbref_id': '6250083a'},
     {'player_id': 406, 'fbref_id': '6c7762c3'},
     {'player_id': 482, 'fbref_id': 'cb4f24c6'},
     {'player_id': 521, 'fbref_id': '88ced02c'},
     {'player_id': 564, 'fbref_id': '8b529245'},
    
     {'player_id': 414, 'fbref_id': '5540c9dc'},
     {'player_id': 416, 'fbref_id': '5e508d0d'},
     {'player_id': 419, 'fbref_id': 'd6192210'},
     {'player_id': 420, 'fbref_id': '76b5cbbd'},
     {'player_id': 499, 'fbref_id': '9f30e7ed'},
     {'player_id': 621, 'fbref_id': '45787ffa'},
     {'player_id': 639, 'fbref_id': '165cf989'},
    
     {'player_id': 429, 'fbref_id': '07666c20'},
     {'player_id': 433, 'fbref_id': 'ba3ca78f'},
     {'player_id': 434, 'fbref_id': '9c318325'},
     {'player_id': 440, 'fbref_id': '9f1893b9'},
     {'player_id': 444, 'fbref_id': 'a36d4785'},
     {'player_id': 447, 'fbref_id': '2b1c4abd'},
     {'player_id': 449, 'fbref_id': '6613c819'},
     {'player_id': 637, 'fbref_id': '95827107'},
     {'player_id': 638, 'fbref_id': 'dcba03f6'},
    
     {'player_id': 453, 'fbref_id': 'da8b19e2'},
     {'player_id': 454, 'fbref_id': 'f5a00fa4'},
     {'player_id': 455, 'fbref_id': '79b0d6a0'},
     {'player_id': 456, 'fbref_id': '0bad4516'},
     {'player_id': 462, 'fbref_id': 'e28868f3'},
     {'player_id': 463, 'fbref_id': 'da23d58d'},
     {'player_id': 466, 'fbref_id': '44bfb6c5'},
     {'player_id': 469, 'fbref_id': '7fcc71d8'},
     {'player_id': 471, 'fbref_id': 'b937e490'},
     {'player_id': 472, 'fbref_id': 'a89e7b29'},
     {'player_id': 474, 'fbref_id': '7ba2eaa9'},
     {'player_id': 475, 'fbref_id': '41ff002d'},
     {'player_id': 476, 'fbref_id': 'bf9c0749'},
     {'player_id': 515, 'fbref_id': '3b029691'},
     {'player_id': 546, 'fbref_id': 'd04b94db'},
     {'player_id': 553, 'fbref_id': '012c38ec'},
     {'player_id': 610, 'fbref_id': '25b82f9a'},
     {'player_id': 611, 'fbref_id': '01ba3996'},
     {'player_id': 614, 'fbref_id': 'cc3b8e78'},
     {'player_id': 615, 'fbref_id': 'ae0e6ba7'},
     {'player_id': 616, 'fbref_id': 'a7cf1541'},
     {'player_id': 617, 'fbref_id': '43f9b4ba'},
     {'player_id': 629, 'fbref_id': '17be8dc3'},
    
     {'player_id': 289, 'fbref_id': '1e26e376'},
     {'player_id': 292, 'fbref_id': 'b6964eb6'},
     {'player_id': 294, 'fbref_id': 'e0991c04'},
     {'player_id': 297, 'fbref_id': 'def899bd'},
     {'player_id': 299, 'fbref_id': 'b853e0ad'},
     {'player_id': 301, 'fbref_id': '810e3c74'},
     {'player_id': 302, 'fbref_id': '507c7bdf'},
     {'player_id': 304, 'fbref_id': '6639e500'},
     {'player_id': 314, 'fbref_id': 'd9565625'},
     {'player_id': 316, 'fbref_id': '2696b6a9'},
     {'player_id': 317, 'fbref_id': '4e015693'},
     {'player_id': 319, 'fbref_id': '91748070'},
     {'player_id': 568, 'fbref_id': 'e73c9bb2'},
     {'player_id': 575, 'fbref_id': 'ac7e7b1c'},
     {'player_id': 604, 'fbref_id': '9dab6e3b'},
     {'player_id': 640, 'fbref_id': ''},
]

In [183]:
fpl_player_stats_df.query(f"player_id == {list(unmapped_fpl_ids)}")[['player_id', 'name', 'team']]\
.query("team == 'man_utd'")

,player_id,name,team
380,289,Sergio Romero,man_utd
383,292,Chris Smalling,man_utd
385,294,Marcos Rojo,man_utd
388,297,Phil Jones,man_utd
390,299,Frederico Rodrigues de Paula Santos,man_utd
392,301,Jesse Lingard,man_utd
393,302,Bruno Miguel Borges Fernandes,man_utd
395,304,Andreas Pereira,man_utd
405,314,José Diogo Dalot Teixeira,man_utd
407,316,Tahith Chong,man_utd


In [184]:
len(unmapped_players)

236

In [185]:
(unmapped_fpl_ids).difference(set([pl["player_id"] for pl in unmapped_players]))

set()

In [195]:
unmapped_players_df = pd.DataFrame(unmapped_players)

### Redoing dfs

In [188]:
pre_final_df.shape

(409, 216)

In [189]:
mapped_df.columns

Index(['player_id', 'fbref_id'], dtype='object')

In [196]:
unmapped_players_df.columns

Index(['player_id', 'fbref_id'], dtype='object')

In [198]:
full_player_set = pd.concat([mapped_df, unmapped_players_df])

In [205]:
full_player_set.shape

(641, 2)

In [207]:
full_player_set.to_csv("../../data/off_fpl_fbref_player_mapping.csv", index=False)

In [190]:
fbref_all.shape

(473, 163)

In [191]:
fbref_all[fbref_all.duplicated('fbref_id')]

,name,profile_url,nationality,position,age,minutes_90s_x,goals,shots_total,shots_on_target,shots_on_target_pct,...,interceptions_y,tackles_won_y,pens_won,pens_conceded,own_goals,ball_recoveries,aerials_won,aerials_lost,aerials_won_pct,fbref_id
113,Ross Barkley,https://fbref.com/en/players/3a24769f/Ross-Bar...,"['eng', 'ENG']",MF,27 years and 051 days,0.5,0,1,1,100.0,...,0,0,0,0,0,8,3,1,75.0,3a24769f
264,Theo Walcott,https://fbref.com/en/players/1ecb65be/Theo-Wal...,"['eng', 'ENG']","MF,FW",31 years and 315 days,12.2,2,22,8,36.4,...,6,15,0,0,0,87,4,23,14.8,1ecb65be
374,Robert Snodgrass,https://fbref.com/en/players/9f4647e9/Robert-S...,"['sco', 'SCO']",FW,33 years and 140 days,1.0,0,1,0,0.0,...,1,3,0,0,0,8,1,3,25.0,9f4647e9
434,Ruben Loftus-Cheek,https://fbref.com/en/players/e97fd090/Ruben-Lo...,"['eng', 'ENG']","FW,MF",25 years and 002 days,8.7,1,14,5,35.7,...,7,11,1,0,0,51,17,35,32.7,e97fd090


In [192]:
fbref_all.query('fbref_id == "3a24769f"')

,name,profile_url,nationality,position,age,minutes_90s_x,goals,shots_total,shots_on_target,shots_on_target_pct,...,interceptions_y,tackles_won_y,pens_won,pens_conceded,own_goals,ball_recoveries,aerials_won,aerials_lost,aerials_won_pct,fbref_id
37,Ross Barkley,https://fbref.com/en/players/3a24769f/Ross-Bar...,"['eng', 'ENG']",MF,27 years and 051 days,6.7,2,22,6,27.3,...,2,0,0,0,0,61,10,4,71.4,3a24769f
113,Ross Barkley,https://fbref.com/en/players/3a24769f/Ross-Bar...,"['eng', 'ENG']",MF,27 years and 051 days,0.5,0,1,1,100.0,...,0,0,0,0,0,8,3,1,75.0,3a24769f


In [199]:
combined_one_df = pd.merge(fbref_all, full_player_set, on='fbref_id')

In [200]:
combined_one_df.shape

(471, 164)

In [201]:
combined_two_df = pd.merge(fpl_player_stats_df, full_player_set, on='player_id')

In [202]:
combined_one_df.shape

(471, 164)

In [208]:
combined_one_df.player_id

0      255
1      249
2      243
3      251
4      253
      ... 
466     26
467      5
468     14
469     21
470      7
Name: player_id, Length: 471, dtype: int64

In [203]:
combined_two_df.shape

(641, 54)

In [209]:
combined_two_df.player_id

0        1
1        2
2        3
3        4
4        5
      ... 
636    615
637    616
638    617
639    626
640    629
Name: player_id, Length: 641, dtype: int64

In [212]:
combined_stats = pd.merge(
    combined_two_df, 
    combined_one_df, 
    how="left", 
    on="player_id", 
    suffixes=("_fpl", "_fbref"))

In [214]:
combined_stats.to_csv("../../data/combined_stats.csv", index=False)